<a href="https://colab.research.google.com/github/Muhammad-BILAL-5/simple-multi-agents-workflow/blob/main/Simple_Multi_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langgraph langchain langchain_community langchain-chroma langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.2 MB/s eta 0:00

In [ ]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('Gemini_Api_Key')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("langchai_api_key")
os.environ["LANGCHAIN_PROJECT"] = "langchain-demo"

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://python.langchain.com/docs/tutorials/query_analysis/")
docs = loader.load_and_split()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(docs)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings )

In [ ]:
from langchain.retrievers import MultiQueryRetriever
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-8b")

retriever = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(),
    llm=llm
)

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    db.as_retriever(),
    "langchain_tool",
    "you are an question-answering tool , you have to provide answers to your users precisely and mindfully according to the context provided to you"
)

retriever_tool.run("what is query analysis")



'Page Not Found | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1\uf8ffüí¨SearchPage Not FoundWe could not find what you were looking for.Please contact the owner of the site that linked you to the original URL and let them know their link is broken.CommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright ¬© 2024 LangChain, Inc.'

In [ ]:
!pip  install --upgrade wikipedia arxiv

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.9 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=a518fdb97a674d88e886de43d9bbfc1816af90aaf07304cb33a6453e5f68b288
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=ce48990a40b7ebf2304af04978b73f34271b311665c852ee7b49731c20ded107
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built wikipedia sgmllib3k


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from pydantic import BaseModel, Field


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )


wiki = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=WikipediaAPIWrapper(),
    return_direct=True,
)

print(wiki.run("langchain"))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.



Page: Retrieval-augmented generation
Summary: Retrieval Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment inf

In [ ]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
from pydantic import BaseModel, Field


class ArxivInputs(BaseModel):
    """Inputs to the Arxiv tool."""

    query: str = Field(
        description="query to look up in Arxiv, should be 3 or less words"
    )
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(), args_schema=ArxivInputs)


arxiv.run("agents in Rag")

"Published: 2024-10-17\nTitle: RAG-DDR: Optimizing Retrieval-Augmented Generation Using Differentiable Data Rewards\nAuthors: Xinze Li, Sen Mei, Zhenghao Liu, Yukun Yan, Shuo Wang, Shi Yu, Zheni Zeng, Hao Chen, Ge Yu, Zhiyuan Liu, Maosong Sun, Chenyan Xiong\nSummary: Retrieval-Augmented Generation (RAG) has proven its effectiveness in\nmitigating hallucinations in Large Language Models (LLMs) by retrieving\nknowledge from external resources. To adapt LLMs for RAG pipelines, current\napproaches use instruction tuning to optimize LLMs, improving their ability to\nutilize retrieved knowledge. This supervised fine-tuning (SFT) approach focuses\non equipping LLMs to handle diverse RAG tasks using different instructions.\nHowever, it trains RAG modules to overfit training signals and overlooks the\nvarying data preferences among agents within the RAG system. In this paper, we\npropose a Differentiable Data Rewards (DDR) method, which end-to-end trains RAG\nsystems by aligning data preference

In [ ]:
tools = [retriever_tool, arxiv , wiki]

In [ ]:
from typing import TypedDict , Annotated , List , Sequence
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:
from langchain.agents import create_tool_calling_agent , AgentExecutor
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder


prompt= ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("system", "You are an Agent assistant that have some additional tools , Answer the user queries{input} using these tools and generate natural language response"),

])

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
from langchain.schema import HumanMessage , AIMessage

def agent_executor_node(state):
  input_message = state["messages"][-1].content
  result = agent_executor.invoke({"input": input_message})
  return {"messages": result["output"]}

In [ ]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

def tool_call(state):
  question = state["messages"][-1].content
  result = tool_executor.invoke(question)
  return {"messages": result["output"]}


<ipython-input-35-800b4fe953fb>:3: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


In [ ]:
from langgraph.graph import StateGraph , END

graph = StateGraph(AgentState)

graph.add_node("agent_node" , agent_executor_node)
graph.add_node("tool_node" , tool_call)

graph.set_entry_point("agent_node")
graph.add_edge("agent_node" , "tool_node")
graph.add_edge("agent_node" , END)

app=graph.compile()

In [ ]:
from langchain.schema import BaseMessage , HumanMessage
input = { "messages": [HumanMessage(content="how to make multi agent app using langraph?")]}

for output in app.stream(input, {"recursion_limit":10}):
  for key,value in output.items():
    print(f"Get response from {key}")
    print(value)
    print("----")